### Generative Adversarial Model - Group 9 <div style="text-align: right; float: right"> IANNWTF - Sheet08 </div>
In this sheet we used a generative model on the MNIST data set to generate "fake" handwritten numbers.

The training process is saved as summaries and saved in the `summary_dir` directory (it's called summary...). In tensorboard you can find the graphs for the losses, and can look at the images over the course of training. So please execute code to access graphs.

##### The usual imports

In [151]:
import numpy as np
import struct, os, time
import tensorflow as tf
import json

##### Recipe data script

In [152]:
# %load recipes.py
import pymongo, json, numpy as np
from collections import Counter
from nltk.tokenize import RegexpTokenizer


class Recipes:

  def __init__(
      self,
      useMongo=True,
      config="config.json"
    ):

    with open(config) as cfg:
      self.data = json.load(cfg)
      storage = self.data['storage']
      CRAWL_FOLDER = storage['folder']
      RECIPE_COLLECTION = storage['collection']

    client = pymongo.MongoClient('mongodb://localhost:27017')
    self.db = client['iannwtf']
    self.coll = self.db[RECIPE_COLLECTION]

  def create_dictionaries(self,vocabulary_size=20000):
    agg = self.coll.aggregate([
        {"$project": {"_id": 0, "ingredients.foodId": 1, "ingredients.name": 1}},
        {"$unwind": "$ingredients"},
        {
            "$group": {
                "_id": "$ingredients.foodId",
                "name": {"$first": "$ingredients.name"}
            }
        }
      ], allowDiskUse=True)
    
    

    self._ing2id = {item['name'].lower(): int(item['_id']) for item in agg}
    self._ing2id['miracel whipmyass'] = 1362
    self._ing2id['NO_INGREDIENT'] = 0
    self._id2ing = dict(zip(self._ing2id.values(), self._ing2id.keys()))

    self._word2id = {}
    self._id2word = {} 
    self._unit2id = {}
    self._id2unit = {}

    ingredient_size = len(self._ing2id)
    unit_size = len(self._unit2id)

    return ingredient_size, unit_size, vocabulary_size

  def ings2ids(self, ings):
    return self._2(ings, self._ing2id, 0)
  
  def units2ids(self, units):
    return self._2(units, self._unit2id, 0)

  def words2ids(self, words):
    return self._2(words, self._word2id, 0)

  def ids2ings(self, ids):
    return self._2(ids, self._id2ing, "NO_INGREDIENT")
  
  def ids2units(self, ids):
    return self._2(ids, self._id2unit, "NO_UNIT")

  def ids2words(self, ids):
    return self._2(ids, self._id2word, "UNKNOWN")

  def _2(self, names, d, undef):
    if type(names) in [list, range]:
      return [d.get(n, undef) for n in names]
    elif type(names) == np.ndarray:
      res = np.full_like(names, "", dtype=object)
      for i in range(res.shape[0]):
        res[i] = self._2(names[i], d, undef)
      return res
    else:
      return d.get(names, undef)

  def get_ingredient_batch(self, batchsize=25, max_ingredients=15):
    count = 0
    batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)
    cursor = self.coll.find({}, {"ingredients.foodId": 1, "_id": 0}, no_cursor_timeout = True)

    recipes = np.array(list(cursor))
    np.random.shuffle(recipes)

    for doc in recipes:
      _ids = [int(x['foodId']) for x in doc['ingredients']]
      if len(_ids) > max_ingredients:
          _ids = _ids[:max_ingredients]
      batch[count][:len(_ids)] = _ids[:]
      count += 1

      if count == batchsize:
          yield batch
          count = 0
          batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)


##### Lukas' layer helper file
The `back_conv_layer` was implemented by us. It is mostly analogue to the `conv_layer` function, with the biggest difference being the `target_shape` parameter, which holds the output shape to be genereated by the transposed convolution.

In [153]:
# %load 08_mnist-gan-layers.py
def feed_forward_layer(x, target_size, normalize = False, activation_function = None):
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    if normalize:
        activation = batch_norm(activation, [0])
    
    return activation_function(activation) if callable(activation_function) else activation


def conv_layer(x, kernel_quantity, kernel_size, stride_size, normalize = False, activation_function = False):
    print("Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]
    fan_in = int(x.shape[1] * x.shape[2])
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, depth, kernel_quantity], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    activation = tf.nn.conv2d(x, kernels, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation


def back_conv_layer(x, target_shape, kernel_size, stride_size, normalize = False, activation_function = False):
    print("De-Conv-Layer:" + str(x.shape))
    depth = x.shape[-1]                   #number of activation maps in the de_conv layer (in_channels)
    fan_in = int(x.shape[1] * x.shape[2]) #resolution of the activation map, important for proper intitialization of kernel weights
    kernel_quantity = target_shape[-1]    #number of activation maps in the target layer (out_channel)
    
    if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        
    # switch kernel_quantity and depth (in_channels and out_channels) because we do deconvolution
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, kernel_quantity, depth], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)
    
    # use conv2d_transpose as instructed
    activation = tf.nn.conv2d_transpose(x, kernels, target_shape, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    
    if normalize:
        activation = batch_norm(activation, [0, 1, 2])
    
    return activation_function(activation) if callable(activation_function) else activation

def flatten(x):
    size = int(np.prod(x.shape[1:]))
    return tf.reshape(x, [-1, size])

def _pop_batch_norm(x, pop_mean, pop_var, offset, scale):
    return tf.nn.batch_normalization(x, pop_mean, pop_var, offset, scale, 1e-6)

def _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale):
    decay = 0.99
    
    dependency_1 = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
    dependency_2 = tf.assign(pop_var, pop_var * decay + var * (1 - decay))

    with tf.control_dependencies([dependency_1, dependency_2]):
        return tf.nn.batch_normalization(x, mean, var, offset, scale, 1e-6)

def batch_norm(x, axes):
    depth = x.shape[-1]
    mean, var = tf.nn.moments(x, axes = axes)
    global is_training
    
    var_init = tf.constant_initializer(0.0)
    offset = tf.get_variable("offset", [depth], tf.float32, var_init)
    var_init = tf.constant_initializer(1.0)
    scale = tf.get_variable("scale", [depth], tf.float32, var_init)
    
    pop_mean = tf.get_variable("pop_mean", [depth], initializer = tf.zeros_initializer(), trainable = False)
    pop_var = tf.get_variable("pop_var", [depth], initializer = tf.ones_initializer(), trainable = False)
    
    return tf.cond(
        is_training,
        lambda: _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale),
        lambda: _pop_batch_norm(x, pop_mean, pop_var, offset, scale)
    )


##### Loading recipe data, defining hyperparameters

In [154]:
config = "config.json"
recipes = Recipes(config)

with open(config,"r",encoding="utf-8") as cfg:
    settings = json.load(cfg)["network"]

hyperparams = settings["hyperparameters"]
directories = settings["directories"]

epochs = 2
learning_rate = 0.0004
beta1 = 0.5
batch_size = 50

#gen_feature_maps = [64, 32, 16] # target shapes for generator layers

# the pretrained weights
embedding_weights = directories["embedding_weights"]

# for tensorboard
weight_dir = directories["weight_dir"]
summary_dir = directories["summary_dir"]
     
## Hyperparameters
embedding_size = hyperparams["embedding_size"]
lstm_memory_size = hyperparams["lstm_memory_size"]
dropout_rate = hyperparams["dropout_rate"]
optimizer = tf.train.AdamOptimizer

z_size = hyperparams["z_size"]
max_ingredients = hyperparams["max_ingredients"]

#dis_feature_maps = [ 8, 16, 32] # target shapes for discriminator layers. maybe to be changed? dunno
ingredient_size, unit_size, vocabulary_size = recipes.create_dictionaries()

##### Helper function to retrieve closest ingredient from embedding vector

In [155]:
def get_ings(embeddings, ingredients):
    '''
    embeddings: the weight matrix with all ingredients representations shape: [ingredient_size, lstm_memory_size]
    ingredients: a matrix of ingedient vectors of who we want to find the nearest neigbhours
      
    '''''
    normed_embedding = tf.nn.l2_normalize(embeddings, dim=1)
    if (type(ingredients)!= list):
        arrays = ingredients
    #else: # we shouldn't have IDs, delete later
     #   ingredients = np.array(ingredients)
      #  arrays = tf.nn.embedding_lookup(embeddings, ingredients)
    
    # initalize "recipes". not sure yet how or if the loop thing is even the optimal way to do it
    
    recipes = []
    
    arraylist = tf.unstack(arrays)
    
    for array in arraylist : # get each recipe (sample in batch_size)
        
        normed_array = tf.nn.l2_normalize(array, dim=1) # what is this for?
  
        cosine_similarity = tf.matmul(normed_array, tf.transpose(normed_embedding, [1, 0]))
        
        closest_ings = tf.argmax(cosine_similarity, 1)  # shape [max_ingredients], type int64. we only need 1 ingredient, so no need for top_k anymore
        
        # stack this on top of recipes so we get batch_size, max_ingredients as output size in the end.
        # todo: actually implement.
        recipes = recipes + [closest_ings]
    
    recipes = tf.stack(recipes)
    return recipes # returns indices of closest ingredients. let's hope the dimension stuff works out in this function

In [166]:
def dis_conv(x, window_size) : 
    
    x = tf.reshape(x,[2*batch_size, embedding_size, max_ingredients, 1])
    
    var_init = tf.random_normal_initializer() # how to initialize?
    kernel = tf.get_variable("kernel", [embedding_size, window_size, 1,1], tf.float32, var_init)
    
    y = tf.nn.conv2d(x, kernel, strides = [1, 1, 1, 1], padding = "VALID")    
    new_length = y.get_shape()[2]

    print("New shape after convolution:" + str(y.get_shape()))
    
    var_init = tf.constant_initializer(0.0)
    # I think this might be wrong
    biases = tf.get_variable("biases", [1,1, new_length, 1], initializer = var_init)
    
    y = y + biases
    
    return tf.squeeze(tf.nn.max_pool(y, [1,1, new_length, 1], strides = [1, 1, 1, 1], padding = "VALID"))

##### Defining the graph

In [167]:
# Defining the graph
tf.reset_default_graph()


###### The generator network. ######      
with tf.variable_scope("generator"):
  
    # see homework for original gen network

    ## placeholders
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size],name="input_vec") # for random input
        is_training = tf.placeholder(tf.bool, [],name="is_training")
        
        # what do the dimensions of this denote again?
        # i put *max_ingredients so it doesn't crash. like where I created the BasicLSTMCell a couple of lines later. how many "units" does it need?
        cell_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, lstm_memory_size*max_ingredients],name="hidden_state")
 
        
    ##### rnn
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_memory_size*max_ingredients)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        
        zero_state = cell.zero_state(batch_size, tf.float32)
        #print(tf.shape(zero_state))
        #zero_state = tf.random_normal([batch_size,]) # randomize zero_state
             
        state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state) # should be fine so far? no changes needed?
        
        outputs, state = tf.nn.static_rnn(cell, [input_vec], initial_state = state)
        
        
        # so at least in the example, lstm_memory_size == embedding_size.
        # so what do we do about the subsequence length?
        # for now I'll just ignore since we haven't even used sequences but the input_vec only. veeeery unsure about
        # this though.
        # so here I will state what the output should look like but not sure how we get the LSTM to produce this exact amount of data
        outputs = tf.reshape(tf.concat(outputs, axis=1), [batch_size, max_ingredients, embedding_size])


    ##### embedding
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embed", [ingredient_size, embedding_size], initializer=init, trainable=False)
                
            
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


##### The discriminator network #####

with tf.variable_scope("discriminator"):
    
    
    orig_recipes_placeholder = tf.placeholder(tf.int64, [batch_size,max_ingredients], name="orig_recipes_placeholder")
    orig_recipes = tf.nn.embedding_lookup(embeddings, orig_recipes_placeholder)
    
    readable_outputs = get_ings(embeddings, outputs)
    
    # create input batch for discriminator
    batch = tf.concat([outputs, orig_recipes],axis=0)
    #batch = tf.reduce_mean(batch,axis=2)

    
    with tf.variable_scope("conv1"):
    #    l1 = feed_forward_layer(batch,64,False,None)   
        conv1 = dis_conv(batch,2)
    
    with tf.variable_scope("conv2"):
        conv2 = dis_conv(batch,3)
    
    with tf.variable_scope("conv3"):
        conv3 = dis_conv(batch,5)
    
    with tf.variable_scope("conv4"):
        conv4 = dis_conv(batch,7)
        
    conv_outputs = tf.stack([conv1,conv2,conv3,conv4], axis=1)
    print(conv_outputs.get_shape())
        
    
    with tf.variable_scope("readout"):
        #logits = feed_forward_layer(l1,1,False,activation_function=tf.nn.sigmoid)
        logits = feed_forward_layer(conv_outputs,1,False,None)
                                    
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to compute cross entropy for generator. consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy. first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits
        
    gen_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # tensorboard stuff
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    #tf.summary.scalar("discriminator_loss", dis_loss)
    
    
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    
    # initialize optimizer
    optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)  
    
    # define training steps with respective variable lists
    gen_step = optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    summaries = tf.summary.merge_all()

New shape after convolution:(100, 1, 14, 1)
New shape after convolution:(100, 1, 13, 1)
New shape after convolution:(100, 1, 11, 1)
New shape after convolution:(100, 1, 9, 1)
(100, 4)
Forward-Layer:(100, 4)


##### This is where we train our network

In [168]:
from tensorflow.python.tools import inspect_checkpoint as chkp
from tensorflow.python import pywrap_tensorflow

reader = pywrap_tensorflow.NewCheckpointReader(embed_ckpt)
var_to_shape_map = reader.get_variable_to_shape_map()
for key in sorted(var_to_shape_map):
    print("tensor_name: ", key)
    print(reader.get_tensor(key).shape)
    
print(embeddings.shape)

tensor_name:  ingredient_embedding
(3601, 64)
(3601, 64)


In [170]:
# HAVE ONLY MADE SMALL ADJUSTMENTS. don't think there's a point if we don't have data



with tf.Session() as session:
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None or True:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)    
    
    embed_saver = tf.train.Saver(var_list={'ingredient_embedding': embeddings})
    if os.path.isdir(embedding_weights):
        embed_ckpt = tf.train.latest_checkpoint(embedding_weights)
        if embed_ckpt is not None:
            embed_saver.restore(session, embed_ckpt)
        else:
            raise Exception("Embedding weights not found")
    
    
    ## Train all the epochs and save the summaries for each timestep
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch, end='')
        
        real_data = recipes.get_ingredient_batch(batch_size, max_ingredients) # get batch of original recipes #mnist.get_batch(batch_size)
        t = time.time()
        
        counter = 0
        
        for data in real_data:
            
            _state = session.run(zero_state)
            
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))
            
            # feed data into placeholders
            _feed_dict = {input_vec: z_vec, 
                         orig_recipes_placeholder: data,
                         is_training: True,
                         cell_state: _state.c,
                         hidden_state: _state.h,}
            
            # let's train our network!
            _gStep, _dStep, _state, _summ, _step,\
            _fr, _or, _logits, _dis_loss = session.run([
                                                gen_step, dis_step, state,
                                                summaries, global_step,
                                                readable_outputs, orig_recipes_placeholder,
                                                logits, dis_loss
                                                ],
                                                feed_dict=_feed_dict)
            train_writer.add_summary(_summ, _step)
            
            _fr = recipes.ids2ings(_fr)
            
    
            
            if counter % 100 == 0 :
                
                #for i in range(batch_size) :
                #        print(_fr[i][np.where(_fr[i]!='NO_INGREDIENT')])

                #print(np.stack(_logits,1))
                #print(_fr)
                print(_fr)
                #print(np.stack(_dis_loss,1))
                print("------------------------------------------")
            
            counter = counter + 1
            #print(str(_fr) + str(_fr.shape))
            
        t = time.time() - t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        train_saver.save(session, weight_dir, global_step=_step)

No checkpoint found. Initialising variables.
INFO:tensorflow:Restoring parameters from embedding_weights/model.ckpt
Starting epoch 1...[['senf' 'kartoffel(n)' 'lachs' 'butter' 'oliven' 'sesam' 'sternanis'
  'honig' 'zwiebel(n)' 'sahne' 'zitronensaft' 'haferflocken'
  'limettensaft' 'erbsenschote(n)' 'mehl']
 ['eis' 'milch' 'butter' 'hackfleisch' 'mehl' 'sonnenblumenkerne'
  'schalotte(n)' 'ei(er)' 'crème fraîche' 'gemüsebrühe'
  'lasagneplatte(n)' 'pfeffer' 'butter' 'sahne' 'möhre(n)']
 ['petersilie' 'schmand' 'salz' 'weißkohl' 'leinsamen' 'ei(er)' 'milch'
  'wasser' 'sahne' 'essig' 'natron' 'butter' 'mehl' 'kohlrabi' 'rotwein']
 ['dill' 'möhre(n)' 'tomatenmark' 'salz' 'frühlingszwiebel(n)'
  'currypulver' 'thymian' 'zitrone(n)' 'salz' 'wasser' 'apfelsaft'
  'backpulver' 'butter' 'joghurt' 'quark']
 ['schweineschmalz' 'vanillezucker' 'banane(n)' 'wasser' 'kardamom'
  'paniermehl' 'knoblauch' 'zitronensaft' 'limette(n)' 'butter' 'butter'
  'kartoffel(n)' 'butter' 'salz' 'honig']
 ['butt

[['löffelbiskuits' 'mehl' 'schmelzkäse' 'tomate(n)' 'olivenöl'
  'knoblauch' 'koriander' 'möhre(n)' 'wasser' 'paprikapulver' 'käse'
  'wasser' 'ananas' 'wasser' 'harissa']
 ['hackfleisch' 'crème fraîche' 'limette(n)' 'ananas' 'butter'
  'zitronensaft' 'brötchen' 'mehl' 'butter' 'salz' 'kirschwasser'
  'dorschfilet(s)' 'joghurt' 'tomatenmark' 'mehl']
 ['mascarpone' 'ei(er)' 'margarine' 'salz' 'ei(er)' 'zucchini'
  'zuckerrübensirup' 'schalotte(n)' 'joghurt' 'rinderleber' 'milch'
  'anstellgut' 'zwiebel(n)' 'ei(er)' 'milch']
 ['paprikapulver' 'kartoffel(n)' 'zimt' 'ei(er)' 'möhre(n)' 'speck'
  'rosmarin' 'pinienkerne' 'butter' 'zitrone(n)' 'butterschmalz' 'kapern'
  'wasser' 'zitronensaft' 'zucker']
 ['sahnesteif' 'butter' 'gewürzgurke(n)' 'ei(er)' 'zitronat' 'speck'
  'currypulver' 'ei(er)' 'sahne' 'möhre(n)' 'milch' 'wasser' 'wasser'
  'knoblauch' 'speisestärke']
 ['ei(er)' 'wasser' 'butter' 'ingwer' 'wasser' 'zitronensaft'
  'petersilie' 'rosmarin' 'sahne' 'zitrone(n)' 'glasnudeln' 'b

[['hackfleisch' 'krabbenpaste' 'hefe' 'butter' 'bambussprosse(n)'
  'salatgurke(n)' 'fett' 'salami' 'nelke(n)' 'zitrone(n)' 'rotwein'
  'lorbeerblätter' 'kohlrabi' 'pinienkerne' 'zimt']
 ['ei(er)' 'reis' 'zwiebel(n)' 'erdnüsse' 'butter' 'zitronensaft' 'käse'
  'speisestärke' 'milch' 'salz' 'sonnenblumenkerne' 'salz' 'kartoffel(n)'
  'schalotte(n)' 'knoblauchzehe(n)']
 ['hackfleisch' 'woksauce' 'zimt' 'butter' 'sahnesteif' 'tomate(n)'
  'dill' 'hackfleisch' 'butter' 'salz' 'nelke(n)' 'salatgurke(n)'
  'kohlrabi' 'pinienkerne' 'butter']
 ['mehl' 'butter' 'gewürzgurke(n)' 'ei(er)' 'pfefferkörner' 'zucchini'
  'salzwasser' 'salatgurke(n)' 'milch' 'salz und pfeffer' 'zimt' 'wasser'
  'currypulver' 'joghurt' 'natron']
 ['fleischbrühe' 'dill' 'wasser' 'butter' 'wasser' 'zitronensaft'
  'knoblauch' 'rosmarin' 'butter' 'zitronensaft' 'rotwein' 'mehl'
  'pfeffer' 'salzwasser' 'butter']
 ['milch' 'butter' 'margarine' 'ei(er)' 'nelke(n)' 'spaghetti' 'hefe'
  'schalotte(n)' 'mehl' 'knoblauch' 'zwie

[['ei(er)' 'wildfond' 'hackfleisch' 'ei(er)' 'wasser' 'knoblauch'
  'kürbis(se)' 'fleischextrakt' 'mehl' 'schalotte(n)'
  'frühlingszwiebel(n)' 'gewürzgurke(n)' 'wasser' 'honig' 'salz']
 ['essig' 'joghurt' 'kaiserschote(n)' 'grenadine' 'wasser' 'zwiebel(n)'
  'fett' 'pfeffer' 'nelke(n)' 'salz' 'mayonnaise' 'apfel' 'dill'
  'tomatenmark' 'honig']
 ['ei(er)' 'wasser' 'milch' 'ei(er)' 'wasser' 'tomate(n)' 'hefe'
  'knoblauchzehe(n)' 'milch' 'champignons' 'essig' 'chilischote(n)'
  'kümmel' 'tomate(n)' 'zucker']
 ['gulasch' 'kokosfett' 'dill' 'zimt' 'wasser' 'zitronensaft'
  'petersilie' 'hackfleisch' 'butter' 'zitronensaft' 'kaki(s)' 'apfel'
  'pfeffer' 'zitronensaft' 'butter']
 ['sahne' 'wasser' 'sahne' 'currypulver' 'wasser' 'zwiebel(n)' 'ei(er)'
  'sahne' 'butter' 'salz' 'currypaste' 'butter' 'möhre(n)' 'zwiebel(n)'
  'koriandergrün']
 ['ei(er)' 'bulgur' 'lorbeerblätter' 'butter' 'wasser' 'zitrone(n)'
  'gemüsebrühe' 'rosmarin' 'mehl' 'zitrone(n)' 'kartoffel(n)'
  'salatgurke(n)' 'salz

[['essig' 'reismehl' 'knoblauch' 'blattspinat' 'wasser' 'wasser'
  'backpulver' 'sojasauce' 'sauerteig' 'paprikapulver' 'chilischote(n)'
  'lorbeerblätter' 'koriander' 'tomatenmark' 'limettensaft']
 ['knollensellerie' 'wasser' 'knoblauch' 'ei(er)' 'wasser' 'ei(er)'
  'backpulver' 'möhre(n)' 'butterschmalz' 'salz' 'chilischote(n)'
  'zwiebel(n)' 'kichererbsen' 'paprikapulver' 'zitronensaft']
 ['knoblauch' 'reiswein' 'wasser' 'zimt' 'wasser' 'dill' 'brötchen'
  'salz und pfeffer' 'butter' 'salz' 'zimtpulver' 'oliven' 'sojasauce'
  'schinken' 'butter']
 ['ei(er)' 'kartoffel(n)' 'ananassaft' 'ei(er)' 'salz' 'tomate(n)' 'mehl'
  'schalotte(n)' 'butter' 'schalotte(n)' 'sojasauce' 'gewürzgurke(n)'
  'sojasauce' 'tomate(n)' 'orangenblütenöl']
 ['hackfleisch' 'apfelsaft' 'salz' 'zimt' 'wasser' 'hackfleisch'
  'lasagneplatte(n)' 'pfeffer' 'ei(er)' 'joghurt' 'gewürzgurke(n)'
  'salatgurke(n)' 'pfeffer' 'honig' 'zimt']
 ['speck' 'joghurt' 'oliven' 'kakaopulver' 'wasser' 'zitronensaft'
  'preiselbe

...

[['sahne' 'wasser' 'ananas' 'ei(er)' 'rohrzucker' 'brötchen' 'sahne'
  'frühlingszwiebel(n)' 'salz' 'tortenguss' 'salatgurke(n)' 'salz'
  'zwiebel(n)' 'tomatenmark' 'petersilie']
 ['wasser' 'wasser' 'ananas' 'ei(er)' 'zucker' 'currypulver' 'ei(er)'
  'schalotte(n)' 'ingwerpulver' 'wasser' 'mehl' 'mehl' 'nelke(n)'
  'zucchini' 'petersilie']
 ['zucker' 'haferflocken' 'wasser' 'zitronensaft' 'fett' 'ingwer'
  'ei(er)' 'einmachhilfe' 'oliven' 'rindfleisch' 'zucker' 'honig'
  'kümmel' 'tomatenmark' 'ei(er)']
 ['ei(er)' 'wasser' 'ananas' 'ei(er)' 'wasser' 'tomate(n)' 'mehl'
  'pfeffer' 'schalotte(n)' 'salz' 'öl' 'zimt' 'zitronensaft' 'petersilie'
  'petersilie']
 ['ei(er)' 'wasser' 'wasser' 'ei(er)' 'hefe' 'currypulver' 'butter'
  'currypulver' 'zitrone(n)' 'chilischote(n)' 'margarine' 'kardamom'
  'margarine' 'sahne' 'wasser']
 ['sahne' 'salz' 'wasser' 'backpulver' 'zucker' 'salz und pfeffer'
  'lasagneplatte(n)' 'champignons' 'butter' 'sahne' 'butter' 'grieß'
  'zucker' 'backpulver' 'salz'

[['frischkäse' 'wasser' 'ananas' 'ei(er)' 'wasser' 'tomate(n)' 'sahne'
  'currypulver' 'zitrone(n)' 'sahne' 'salatgurke(n)' 'butterschmalz'
  'sepiatinte' 'tomatenmark' 'petersilie']
 ['knoblauch' 'zimt' 'wasser' 'pfeffer' 'zucker' 'öl' 'joghurt'
  'currypulver' 'butter' 'sahne' 'milch' 'mehl' 'wasser' 'milch' 'wasser']
 ['mehl' 'wasser' 'ananas' 'ei(er)' 'wasser' 'tomate(n)' 'sahne'
  'petersilie' 'ingwerpulver' 'salz' 'zwieback' 'mehl' 'nelke(n)'
  'champignons' 'petersilie']
 ['schafskäse' 'eiweiß' 'ananas' 'ei(er)' 'zucker' 'tomate(n)'
  'hackfleisch' 'petersilie' 'butterschmalz' 'butter' 'salz' 'mehl'
  'sahne' 'dill' 'petersilie']
 ['mehl' 'mehl' 'wasser' 'backpulver' 'speisestärke' 'öl' 'ei(er)'
  'currypulver' 'butter' 'chilischote(n)' 'zitronat' 'ei(er)' 'öl'
  'milch' 'currypulver']
 ['ei(er)' 'wasser' 'ananas' 'ei(er)' 'wasser' 'senf' 'mehl'
  'salatgurke(n)' 'mehl' 'salz und pfeffer' 'milch' 'mehl' 'ingwer'
  'tomate(n)' 'petersilie']
 ['mehl' 'wasser' 'wasser' 'ei(er)' 'zu

[['sahne' 'wasser' 'wasser' 'backpulver' 'paniermehl' 'öl' 'milch'
  'apfel' 'zitrone(n)' 'kidneybohnen' 'butter' 'gorgonzola' 'naturreis'
  'sahne' 'wasser']
 ['ei(er)' 'ingwer' 'ananas' 'ei(er)' 'zucker' 'tomate(n)' 'ei(er)'
  'petersilie' 'wasser' 'milch' 'öl' 'butterschmalz' 'möhre(n)' 'kümmel'
  'petersilie']
 ['zitronensaft' 'zucker' 'bohnenkraut' 'zucker' 'schmand' 'sojasauce'
  'mehl' 'zimt' 'zitrone(n)' 'wasser' 'nelkenpulver' 'mehl' 'margarine'
  'sahne' 'wasser']
 ['sahne' 'zimt' 'wasser' 'ei(er)' 'zucker' 'öl' 'sahne' 'currypulver'
  'butter' 'rindfleisch' 'butter' 'cima di rapa' 'margarine' 'sahne'
  'saure sahne']
 ['ei(er)' 'wasser' 'gewürzgurke(n)' 'salz' 'zucker' 'currypulver'
  'zucker' 'feldsalat' 'butter' 'kartoffel(n)' 'wasser' 'currypulver'
  'schweineschmalz' 'tomatenmark' 'essig']
 ['ei(er)' 'wasser' 'gewürzgurke(n)' 'backpulver' 'wasser' 'tomate(n)'
  'schmand' 'currypulver' 'butter' 'sahne' 'margarine' 'mehl' 'nelke(n)'
  'champignons' 'knoblauch']
 ['sahne' '

[['sahne' 'wasser' 'joghurt' 'backpulver' 'sahne' 'ananassaft' 'ei(er)'
  'gewürzgurke(n)' 'pinienkerne' 'semmelbrösel' 'gewürzgurke(n)' 'muskat'
  'rosmarin' 'sahne' 'tomate(n)']
 ['ei(er)' 'wasser' 'wasser' 'hefe' 'sahne' 'pfeffer' 'fett' 'zwiebel(n)'
  'butter' 'butter' 'sojasauce' 'kartoffel(n)' 'möhre(n)' 'thymian'
  'currypulver']
 ['ei(er)' 'kartoffel(n)' 'cognac' 'roggenmehl' 'ei(er)' 'pfeffer'
  'butter' 'käse' 'butter' 'wasser' 'apfel' 'joghurt' 'schlagsahne'
  'wasser' 'champignons']
 ['ei(er)' 'wasser' 'dill' 'hefe' 'banane(n)' 'sojasauce' 'milch'
  'petersilie' 'zitrone(n)' 'salz' 'margarine' 'mehl' 'rhabarber'
  'butter' 'salz']
 ['sahne' 'koriandergrün' 'eiweiß' 'ei(er)' 'wasser' 'tomate(n)' 'ei(er)'
  'käse' 'zitrone(n)' 'wasser' 'gewürzgurke(n)' 'vanillearoma' 'konfekt'
  'wasser' 'petersilie']
 ['sahne' 'möhre(n)' 'zucker' 'backpulver' 'wasser' 'pfeffer' 'milch'
  'currypulver' 'butter' 'wacholderbeere(n)' 'honig' 'ei(er)' 'margarine'
  'milch' 'wasser']
 ['ei(er)' 's

KeyboardInterrupt: 